In [1]:
from scraping_utils import *

In [3]:
ls

csv_data/                    image_utils.pyc      scraping_utils.py*
data_20180129.csv            __init__.py*         sold_homes_scrape.py*
data_20180207.csv            load_properties.py*  Untitled.ipynb
image_filter_and_upload.py*  __pycache__/
image_utils.py*              remax_scrape.py*


In [4]:
cd csv_data/

/home/sanjeev/homescan/utils/csv_data


In [5]:
ls

data_20180123.csv  data_20180129.csv  soldhomedata_20180131.csv


In [7]:
import pandas as pd
df = pd.read_csv('soldhomedata_20180131.csv')

In [9]:
df.head()

addr = '100-Long-Creek-Pkwy-Charlotte-NC-28214'

In [11]:
addr in df['Unnamed: 0']

False

In [14]:
addr = df['Unnamed: 0'][0]

In [19]:
df['Unnamed: 0'].str.contains(addr).sum()

1

In [20]:
newaddr = 'testest'

In [23]:
df['Unnamed: 0'].str.contains(addr).sum() >= 1

True